# Use GloVe embeddings of word meanings

In [1]:
import pandas as pd
from toxic_funcs import Cleaner
import numpy as np

In [2]:
#reading only first n for testing purposes
n = 10000
df = pd.read_csv("data/train.csv").set_index('id')
df = df.head(n)

In [3]:
#running through the custom cleaner
from re import subn

clean_comments = []
for comment in df.comment_text.values:
    cleaned, _ = subn('[(){[]}:;.,]', '', Cleaner(comment))
    clean_comments.append(cleaned)


In [4]:
#slapping it onto df

df['cleaned'] = clean_comments

In [5]:
import os
import sys

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids

texts = df.cleaned.values
labels_index = {0: 0, 1:1}
labels = df.toxic.values


In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT =.3

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
X_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]


Using TensorFlow backend.
c:\users\nstein\appdata\local\conda\conda\envs\pydata\lib\site-packages\keras\preprocessing\text.py:157: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 36625 unique tokens.
Shape of data tensor: (10000, 1000)
Shape of label tensor: (10000, 2)


In [7]:
embeddings_index = {}
GLOVE_DIR = os.path.join('..','TEXT_DATA_DIR', 'glove.6B')
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


Found 400000 word vectors.


In [8]:
EMBEDDING_DIM  = 100

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


In [9]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)


## CNN with GloVe
https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

In [14]:
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.models import Model

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

x = Conv1D(256, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)  # global max pooling
x = Flatten()(x)
print(x)
x = Dense(128, activation='relu')(x)
x = Dropout(.2)(x) 
x = Dense(64, activation='relu')(x)
x = Dense(12, activation='relu')(x)



preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

Tensor("flatten_3/Reshape:0", shape=(?, ?), dtype=float32)


In [15]:
# happy learning!
model.fit(X_train, y_train, validation_data=(X_test, y_test),
          epochs=10, batch_size=128)

Train on 7000 samples, validate on 3000 samples
Epoch 1/10
7000/7000 [==============================] - 99s 14ms/step - loss: 0.3909 - acc: 0.9054 - val_loss: 0.3350 - val_acc: 0.8957
Epoch 2/10
7000/7000 [==============================] - 106s 15ms/step - loss: 0.3208 - acc: 0.9060 - val_loss: 0.3341 - val_acc: 0.8957
Epoch 3/10
7000/7000 [==============================] - 106s 15ms/step - loss: 0.3169 - acc: 0.9059 - val_loss: 0.3330 - val_acc: 0.8957
Epoch 4/10
7000/7000 [==============================] - 95s 14ms/step - loss: 0.3129 - acc: 0.9064 - val_loss: 0.3277 - val_acc: 0.8963
Epoch 5/10
7000/7000 [==============================] - 94s 13ms/step - loss: 0.3104 - acc: 0.9069 - val_loss: 0.3716 - val_acc: 0.8963
Epoch 6/10
7000/7000 [==============================] - 94s 13ms/step - loss: 0.3037 - acc: 0.9073 - val_loss: 0.3522 - val_acc: 0.8963
Epoch 7/10
7000/7000 [==============================] - 95s 14ms/step - loss: 0.3036 - acc: 0.9070 - val_loss: 0.3472 - val_acc: 0.896

In [21]:
model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Normal Neural Network Test - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network Test - Loss: 0.7007125447591146, Accuracy: 0.35399999992052716


## LSTM RNN
https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [20]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding


sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 100)         3662600   
_________________________________________________________________
dropout_10 (Dropout)         (None, 1000, 100)         0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_11 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 202       
Total params: 3,753,302
Trainable params: 90,702
Non-trainable params: 3,662,600
_____________________________________________________________

In [28]:
model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Normal Neural Network Test - Loss: {model_loss}, Accuracy: {model_accuracy}")

92

# vanilla bag of words

In [37]:
# Create a Neural Network model here
# YOUR CODE HERE
model = Sequential()
model.add(Dense(1000,activation = "relu", input_dim = 10000))
model.add(Dropout(.2))
model.add(Dense(100, activation = "relu"))
model.add(Dense(10))
model.add(Dense(2, activation = "softmax"))

In [38]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical


X = tokenizer.texts_to_matrix(df.cleaned.values)
y = to_categorical(df.toxic.values)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)


model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)
model.fit(X_train, y_train, epochs = 10, verbose=1)

Epoch 1/10
7500/7500 [==============================] - 34s 5ms/step - loss: 0.2603 - acc: 0.9145
Epoch 2/10
7500/7500 [==============================] - 34s 4ms/step - loss: 0.0982 - acc: 0.9664
Epoch 3/10
7500/7500 [==============================] - 35s 5ms/step - loss: 0.0261 - acc: 0.9943
Epoch 4/10
7500/7500 [==============================] - 35s 5ms/step - loss: 0.0152 - acc: 0.9987
Epoch 5/10
7500/7500 [==============================] - 35s 5ms/step - loss: 0.0118 - acc: 0.9991
Epoch 6/10
7500/7500 [==============================] - 35s 5ms/step - loss: 0.0118 - acc: 0.9991
Epoch 7/10
7500/7500 [==============================] - 35s 5ms/step - loss: 0.0110 - acc: 0.9993
Epoch 8/10
7500/7500 [==============================] - 34s 5ms/step - loss: 0.0111 - acc: 0.9992
Epoch 9/10
7500/7500 [==============================] - 36s 5ms/step - loss: 0.0109 - acc: 0.9993
Epoch 10/10
7500/7500 [==============================] - 34s 5ms/step - loss: 0.0116 - acc: 0.9991


In [39]:
model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Normal Neural Network Test - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network Test - Loss: 0.45488132861237973, Accuracy: 0.9496
